In [2]:
from textstat.textstat import textstatistics, easy_word_set, legacy_round 
import spacy, csv 
# from __future__ import unicode_literals, print_function
import nltk
import re
import io
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import PhraseMatcher

## Readability metrics

In [3]:
# Splits the text into sentences, using 
# Spacy's sentence segmentation which can 
# be found at https://spacy.io/usage/spacy-101 
def break_sentences(text): 
    nlp = spacy.load('en') 
    doc = nlp(text) 
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences 

In [4]:
# Returns Number of Words in the text 
def word_count(text): 
    sentences = break_sentences(text) 
    words = 0
    for sentence in sentences: 
        words += len([token for token in sentence]) 
    return words 

In [5]:
# Returns the number of sentences in the text 
def sentence_count(text): 
    sentences = break_sentences(text) 
    return len(sentences) 

In [6]:
# Returns average sentence length 
def avg_sentence_length(text): 
    words = word_count(text) 
    sentences = sentence_count(text) 
    average_sentence_length = float(words / sentences) 
    return average_sentence_length 

In [7]:
# Textstat is a python package, to calculate statistics from 
# text to determine readability, 
# complexity and grade level of a particular corpus. 
# Package can be found at https://pypi.python.org/pypi/textstat 
def syllables_count(word): 
    return textstatistics().syllable_count(str(word)) 

In [8]:
# Returns the average number of syllables per 
# word in the text 
def avg_syllables_per_word(text): 
    syllable = syllables_count(text) 
    words = word_count(text) 
    ASPW = float(syllable) / float(words) 
    return legacy_round(ASPW, 1) 

In [9]:
# Return total Difficult Words in a text 
def difficult_words(text): 

    # Find all words in the text 
    words = [] 
    sentences = break_sentences(text) 
    for sentence in sentences: 
        words += [token for token in sentence] 

    # difficult words are those with syllables >= 2 
    # easy_word_set is provide by Textstat as 
    # a list of common words 
    diff_words_set = set() 
    
    for word in words: 
        syllable_count = syllables_count(word) 
        if word not in easy_word_set and syllable_count >= 2: 
            diff_words_set.add(word) 

    return len(diff_words_set) 

In [10]:
# A word is polysyllablic if it has more than 3 syllables 
# this functions returns the number of all such words 
# present in the text 
def poly_syllable_count(text): 
    count = 0
    words = [] 
    sentences = break_sentences(text) 
    for sentence in sentences: 
        words += [token for token in sentence] 


    for word in words: 
        syllable_count = syllables_count(word)
        if syllable_count >= 3: 
            count += 1
    return count 

In [11]:
def flesch_reading_ease(text): 
    """ 
        Implements Flesch Formula: 
        Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW) 
        Here, 
        ASL = average sentence length (number of words 
            divided by number of sentences) 
            ASW = average word length in syllables (number of syllables 
            divided by number of words) 
    """
    FRE = 206.835 - float(1.015 * avg_sentence_length(text)) -float(84.6 * avg_syllables_per_word(text)) 
    return legacy_round(FRE, 2) 

In [12]:
def gunning_fog(text): 
    per_diff_words = (difficult_words(text) / word_count(text) * 100) + 5
    grade = 0.4 * (avg_sentence_length(text) + per_diff_words) 
    return grade 

In [13]:
def smog_index(text): 
    """ 
        Implements SMOG Formula / Grading 
        SMOG grading = 3 + ?polysyllable count. 
        Here, polysyllable count = number of words of more 
        than two syllables in a sample of 30 sentences. 
    """

    if sentence_count(text) >= 3: 
        poly_syllab = poly_syllable_count(text) 
        SMOG = (1.043 * (30*(poly_syllab / sentence_count(text)))**0.5) + 3.1291
        return legacy_round(SMOG, 1) 
    else: 
        return 0

In [14]:
def dale_chall_readability_score(text): 
    """ 
        Implements Dale Challe Formula: 
        Raw score = 0.1579*(PDW) + 0.0496*(ASL) + 3.6365 
        Here, PDW = Percentage of difficult words. ASL = Average sentence length 
    """
    words = word_count(text) 
    # Number of words not termed as difficult words 
    count = words - difficult_words(text) 
    if words > 0: 

        # Percentage of words not on difficult word list 
        per = float(count) / float(words) * 100

    # diff_words stores percentage of difficult words 
    diff_words = 100 - per 

    raw_score = (0.1579 * diff_words) + (0.0496 * avg_sentence_length(text)) 

    # If Percentage of Difficult Words is greater than 5 %, then; 
    # Adjusted Score = Raw Score + 3.6365, 
    # otherwise Adjusted Score = Raw Score 

    if diff_words > 5:

        raw_score += 3.6365

    return legacy_round(raw_score, 2) 

In [16]:
dale_chall_readability_score(text)

NameError: name 'text' is not defined

In [17]:
flesch_reading_ease(text)

NameError: name 'text' is not defined

In [93]:
smog_index(text)

3.1

In [94]:
gunning_fog(text)

37.10154335677305

In [95]:
avg_sentence_length(text)

87.74257425742574

## Example: Stemming with NLTK

In [133]:
ps = PorterStemmer()
nltk.download('punkt')
nlp = spacy.blank('en')

bFound= False
target = ps.stem('cookies')
words = word_tokenize(new_text)
for w in words:
    if ps.stem(w)==target:
     bFound = True  
     print(bFound)
     break

## Example: Phrasematcher with spaCy

In [1]:
matcher = PhraseMatcher(nlp.vocab)
terminology_list = ['Barack Obama', 'Angela Merkel', 'Washington, D.C.']
patterns = [nlp(text) for text in terminology_list]
matcher.add('TerminologyList', None, *patterns)

doc = nlp(u"German Chancellor Angela Merkel and US President Barack"
          u"converse in the Oval Office inside the White House in Washington, D.C.")
matches = matcher(doc)
print(matches)

NameError: name 'PhraseMatcher' is not defined

## Example: how to normalize the features

In [307]:
from sklearn import preprocessing

x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)